In [1]:
import pandas as pd
from pandasql import sqldf
import boto3
import json
from keras.callbacks import EarlyStopping, TensorBoard
import os
from geopy.distance import distance
import numpy as np
from sklearn.preprocessing import Normalizer
from keras import models, layers
# # from keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense, Embedding, Flatten, Conv1D, MaxPool1D, Dropout
# # from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt
# import copy
# import numpy as np
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
%run '../extra_fns.ipynb'


Using TensorFlow backend.


In [2]:
with open('../config.json') as json_data:
    config = json.load(json_data)

In [3]:
s3 = boto3.client(
    's3',
    aws_access_key_id=config['boto']['aws_access_key_id'],
    aws_secret_access_key=config['boto']['aws_secret_access_key']
)

In [4]:
callbacks = [
    EarlyStopping(monitor='acc', patience=1),
    TensorBoard(log_dir=config['tensorboard_logs'])
]

In [5]:
model_dir = 'trained_models'
try:
    os.makedirs(model_dir)
except Exception as e:
    print(e)

[Errno 17] File exists: 'trained_models'


In [6]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
train_file = data_dir + '/train.csv'
test_file = data_dir + '/test.csv'

# s3.upload_file(src_file, boto_config['buckets']['kaggle'], src_file)
# s3.upload_file(src_file_cleaned, boto_config['buckets']['kaggle'], src_file_cleaned)

# s3.download_file(boto_config['buckets']['kaggle'], src_file, src_file)
# s3.download_file(boto_config['buckets']['kaggle'], src_file_cleaned, src_file_cleaned)

# train = pd.read_csv(train_file)
# test = pd.read_csv(test_file)

In [7]:
# train['distance'] = train.apply(lambda row : int(distance(
#     (row['pickup_latitude'], row['pickup_longitude']),
#     (row['dropoff_latitude'], row['dropoff_longitude'])).m), axis=1)

# train.pickup_datetime = pd.to_datetime(train.pickup_datetime)

# train['pickup_monthday'] = train.pickup_datetime.dt.day
# train['pickup_weekday'] = train.pickup_datetime.dt.weekday
# train['pickup_hour'] = train.pickup_datetime.dt.hour

In [8]:
# test['distance'] = test.apply(lambda row : int(distance(
#     (row['pickup_latitude'], row['pickup_longitude']),
#     (row['dropoff_latitude'], row['dropoff_longitude'])).m), axis=1)

# test.pickup_datetime = pd.to_datetime(test.pickup_datetime)

# test['pickup_monthday'] = test.pickup_datetime.dt.day
# test['pickup_weekday'] = test.pickup_datetime.dt.weekday
# test['pickup_hour'] = test.pickup_datetime.dt.hour

In [9]:
# train.to_csv('./data/train-clean.csv')
# test.to_csv('./data/test-clean.csv')

train = pd.read_csv('./data/train-clean.csv')
test = pd.read_csv('./data/test-clean.csv')

In [10]:
y_train = np.array(train.trip_duration)
X_train = np.array(train.loc[:, ['passenger_count', 'distance', 'pickup_monthday', 'pickup_weekday', 'pickup_hour']])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

X_test = np.array(test.loc[:, ['passenger_count', 'distance', 'pickup_monthday', 'pickup_weekday', 'pickup_hour']])

In [11]:
nml = Normalizer()

In [12]:
X_train = nml.fit_transform(X_train)
X_val = nml.transform(X_val)
X_test = nml.transform(X_test)

In [14]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))

model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

model.summary()

history = model.fit(X_train, 
                    y_train, 
                    epochs=20, 
                    batch_size=256, 
                    validation_data=(X_val, y_val))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                384       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 449
Trainable params: 449
Non-trainable params: 0
_________________________________________________________________
Train on 1166915 samples, validate on 291729 samples
Epoch 1/20


InternalError: GPU sync failed

In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))

model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

model.summary()

history = model.fit(train_data, 
                    train_targets, 
                    epochs=20, 
                    batch_size=256)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 961
Trainable params: 961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


InternalError: Blas GEMM launch failed : a.shape=(256, 13), b.shape=(13, 64), m=256, n=64, k=13
	 [[{{node dense_1/MatMul}} = MatMul[T=DT_FLOAT, _class=["loc:@training/RMSprop/gradients/dense_1/MatMul_grad/MatMul_1"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_dense_1_input_0_0/_29, dense_1/kernel/read)]]
	 [[{{node loss/mul/_49}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_279_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]